In [1]:
import os
import pickle
import torch
import numpy as np
from model import SentimentLSTM
from dataset import SentimentAnalysisDataset, LABEL_MAP

data_path = 'data'

print('====== TEST =======')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

print('-> Loading vocabulary')
with open('vocab.dict', 'rb') as pkl:
    vocab = pickle.load(pkl)

print('-> Loading datasets')
test_dataset = SentimentAnalysisDataset(os.path.join(data_path, 'testEmotions.csv'), vocab)
train_size = len(test_dataset)

print('-> Initalizing model')
model = SentimentLSTM(len(vocab))
model.to(device)

loss = torch.nn.CrossEntropyLoss()

/Users/snirlugassy/miniconda3/envs/deep/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


====== TEST =======
device: cpu
-> Loading vocabulary
-> Loading datasets


TypeError: SentimentAnalysisDataset.__init__() takes 2 positional arguments but 3 were given

In [10]:
def run_test(model, dataset, device)

    test_loss = 0.0
    y_true = []
    y_predict = []

    model.eval()
    with torch.no_grad():
        for i, (tokens, label) in enumerate(dataset):
            tokens = tokens.to(device)
            label = label.to(device)
            y_true.append(int(label.argmax(dim=0)))

            if len(tokens) == 0:
                # Predict neutral if no token after processing 
                # e.g., only stopwords in the original text
                y_predict.append(int(LABEL_MAP['neutral']))
                continue

            # Forward pass
            y_prob = model(tokens)[-1]
            y_predict.append(int(torch.softmax(y_prob, dim=0).argmax(dim=0)))
            L = loss(y_prob.view(1,-1), label.view(1,-1))
            test_loss += L.item()

    y_true = np.array(y_true)
    y_predict = np.array(y_predict)

    accuracy = sum(y_true == y_predict) / len(y_true)

    return accuracy, test_loss

In [11]:
y_true.shape, y_predict.shape

((4837,), (4837,))

In [12]:
accuracy = sum(y_true == y_predict) / len(y_true)
accuracy

0.26834814967955345